In [1]:
import pandas as pd
import numpy as np
import feather
import os as os
import glob
from joblib import Parallel, delayed
import multiprocessing

In [2]:
GFW_DIR = '/data2/GFW_point/'

In [3]:
gfw_vessel_dat = pd.read_csv('~/Data/GFW_public/fishing_vessels/fishing_vessels.csv')
dat = pd.read_csv('/data2/GFW_point/2016-01-01/messages-2016-01-01-000000000000.csv')
#gfw_identities = gfw_identities.rename(index=str, columns={"ssvid": "mmsi"})
#gfw_identities['year'] = pd.DatetimeIndex(gfw_identities['timestamp']).year 
#gfw_identities['month'] = pd.DatetimeIndex(gfw_identities['timestamp']).month
#gfw_identities['day'] = pd.DatetimeIndex(gfw_identities['timestamp']).day

/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
def data_step(data): 
    # Patagonia shelf coordinates
    #lower left lat: -58
    #lower left lon: -77
    #upper right lat: -23
    #upper right lon: -22
    
    lon1 = -77
    lon2 = -22
    lat1 = -58
    lat2 = -23
    retdat = data[(data['lon'] >= lon1) & (data['lon'] <= lon2) & (data['lat'] >= lat1) & (data['lat'] <= lat2)]
    
    # Separate Year, month, day, hour, minute, second
    retdat.loc[:, 'timestamp'] = pd.to_datetime(retdat['timestamp'], format="%Y-%m-%d %H:%M:%S UTC")
    retdat.loc[:, 'year'] = pd.DatetimeIndex(retdat['timestamp']).year 
    retdat.loc[:, 'month'] = pd.DatetimeIndex(retdat['timestamp']).month
    retdat.loc[:, 'day'] = pd.DatetimeIndex(retdat['timestamp']).day
    retdat.loc[:, 'hour'] = pd.DatetimeIndex(retdat['timestamp']).hour
    retdat.loc[:, 'minute'] = pd.DatetimeIndex(retdat['timestamp']).minute
    retdat.loc[:, 'second'] = pd.DatetimeIndex(retdat['timestamp']).second
    
    # Merge GFW ID data
    retdat = pd.merge(retdat, gfw_vessel_dat, how='left', on='mmsi')  
    
    retdat = retdat[['timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second', 'mmsi', 'lat', 'lon', \
                    'segment_id', 'message_id', 'type', 'speed', 'course', 'heading', 'shipname', 'callsign', \
                     'destination', 'elevation_m', 'distance_from_shore_m', 'distance_from_port_m', 'nnet_score', \
                     'logistic_score', 'flag', 'geartype', 'length', 'tonnage', 'engine_power', 'active_2012', \
                     'active_2013', 'active_2014', 'active_2015', 'active_2016']]
    
    return retdat

In [8]:
def GFW_directories():
    
    dirs = os.listdir(GFW_DIR)
    # Remove subfolders 'BK' and 'identities'
    if 'BK' in dirs:
        dirs.remove('BK')
    
    if 'identities' in dirs:
        dirs.remove('identities')
    
    return dirs

In [47]:
ndat = data_step(data=dat, filename='test2')

TypeError: data_step() got an unexpected keyword argument 'filename'

In [80]:
    data = dat
    lon1 = -77
    lon2 = -22
    lat1 = -58
    lat2 = -23
    retdat = data[(data['lon'] >= lon1) & (data['lon'] <= lon2) & (data['lat'] >= lat1) & (data['lat'] <= lat2)]
    
    # Separate Year, month, day, hour, minute, second
    retdat.loc[:, 'timestamp'] = pd.to_datetime(retdat['timestamp'], format="%Y-%m-%d %H:%M:%S UTC")
    retdat.loc[:, 'year'] = pd.DatetimeIndex(retdat['timestamp']).year 
    retdat.loc[:, 'month'] = pd.DatetimeIndex(retdat['timestamp']).month
    retdat.loc[:, 'day'] = pd.DatetimeIndex(retdat['timestamp']).day
    retdat.loc[:, 'hour'] = pd.DatetimeIndex(retdat['timestamp']).hour
    retdat.loc[:, 'minute'] = pd.DatetimeIndex(retdat['timestamp']).minute
    retdat.loc[:, 'second'] = pd.DatetimeIndex(retdat['timestamp']).second
    
    # Merge GFW ID data
    retdat = pd.merge(retdat, gfw_vessel_dat, how='left', on='mmsi')  
    
    retdat = retdat[['timestamp', \
                     'year', 'month', 'day', 'hour', 'minute', 'second', \
                     'mmsi', 'lat', 'lon', \
                    'segment_id', 'message_id', 'type', 'speed', 'course', 'heading', 'shipname', 'callsign', \
                     'destination', 'elevation_m', 'distance_from_shore_m', 'distance_from_port_m', 'nnet_score', \
                     'logistic_score', 'flag', 'geartype', 'length', 'tonnage', 'engine_power', 'active_2012', \
                     'active_2013', 'active_2014', 'active_2015', 'active_2016']]
    

In [82]:
retdat.day

0         3
1         3
2         3
3         3
4         3
5         3
6         3
7         3
8         3
9         3
10        3
11        3
12        3
13        3
14        3
15        3
16        3
17        3
18        3
19        3
20        3
21        3
22        3
23        3
24        3
25        3
26        3
27        3
28        3
29        3
         ..
326713    3
326714    3
326715    3
326716    3
326717    3
326718    3
326719    3
326720    3
326721    3
326722    3
326723    3
326724    3
326725    3
326726    3
326727    3
326728    3
326729    3
326730    3
326731    3
326732    3
326733    3
326734    3
326735    3
326736    3
326737    3
326738    3
326739    3
326740    3
326741    3
326742    3
Name: day, Length: 326743, dtype: int64

In [34]:
filename = f"{ndat['year'][1]}-" + f"{ndat['month'][1]}".zfill(2) + f"-" + f"{ndat['day'][1]}".zfill(2)

In [45]:
ndat.to_csv('~/Data/test.csv')

In [65]:
ndat.timestamp

0       2016-01-02 03:09:51+00:00
1       2016-01-02 14:05:23+00:00
2       2016-01-02 15:01:15+00:00
3       2016-01-02 16:35:31+00:00
4       2016-01-02 06:11:47+00:00
5       2016-01-02 04:40:14+00:00
6       2016-01-02 06:09:35+00:00
7       2016-01-02 15:42:36+00:00
8       2016-01-02 21:45:30+00:00
9       2016-01-02 12:32:38+00:00
10      2016-01-02 14:07:11+00:00
11      2016-01-02 16:26:18+00:00
12      2016-01-02 06:07:07+00:00
13      2016-01-02 15:46:57+00:00
14      2016-01-02 17:15:31+00:00
15      2016-01-02 17:56:33+00:00
16      2016-01-02 04:35:48+00:00
17      2016-01-02 02:15:21+00:00
18      2016-01-02 02:09:23+00:00
19      2016-01-02 16:30:37+00:00
20      2016-01-02 02:15:05+00:00
21      2016-01-02 01:36:47+00:00
22      2016-01-02 12:27:42+00:00
23      2016-01-02 10:48:44+00:00
24      2016-01-02 15:00:26+00:00
25      2016-01-02 14:02:16+00:00
26      2016-01-02 04:40:15+00:00
27      2016-01-02 04:37:15+00:00
28      2016-01-02 12:28:59+00:00
29      2016-0

In [38]:
dirs = sorted(GFW_directories())

for i in dirs[1:2]:  # Change!!!

    # Get subdirectory list of files
    subdir = GFW_DIR + i
    allFiles = glob.glob(subdir + "/*.csv")
    list_ = []
    
    # Append files in subdir
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
        dat = pd.concat(list_, axis = 0, ignore_index = True)
    
    # Append data
    outdat = data_step(data=dat)
    outdat = outdat.fillna()

    # Get string for filename from timestamp
    filename = f"{outdat['year'][1]}-" + f"{outdat['month'][1]}".zfill(2) + f"-" + f"{outdat['day'][1]}".zfill(2)
    
    # Save unique mmsi for each day
    unique_mmsi_data = outdat['mmsi'].unique()
    unique_mmsi = pd.DataFrame({'mmsi':unique_mmsi_data})
    unique_mmsi.to_feather('~/Data/GFW_point/Patagonia_Shelf/vessel_list/' + filename +  '_vessel_list'  + '.feather')
    
    # Save data
    outdat.to_csv('~/Data/GFW_point/Patagonia_Shelf/csv/' + filename + '.csv', index=False)
    outdat.to_feather('~/Data/GFW_point/Patagonia_Shelf/feather/' + filename + '.feather')
    print(i)

/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/server/pi/homes/woodilla/.conda/envs/baseDS_env/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

2016-01-02


In [232]:
inputs = dirs[3:23]
def processGFW(i):
    subdir = GFW_DIR + i
    allFiles = glob.glob(subdir + "/*.csv")
    list_ = []
    
    # Append files in subdir
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
        dat = pd.concat(list_, axis = 0, ignore_index = True)
    outdat = data_step(data=dat, filename=i)
    outdat.to_csv('~/Data/GFW_point/Patagonia_Shelf/csv/' + i + '.csv')
    outdat.to_feather('~/Data/GFW_point/Patagonia_Shelf/feather/' + i + '.feather')
    #print(i)  
 
num_cores = 20
results = Parallel(n_jobs=num_cores)(delayed(processGFW)(i) for i in inputs)

In [230]:
dirs[1:3]

['2016-01-02', '2016-01-03']